In [92]:
# data import for 2005 data
import pandas as pd

data_2005 = pd.read_csv('/Users/colden/Documents/VSCode/Emos_Classification/Environment_Data_2005_cleaned.csv')

data_2005['description'] = data_2005['name'] + ' ' + data_2005['mission'].fillna('')
data_2005['data_year'] = 2005

#data_2005 = data_2005.drop(data_2005[(data_2005['ntee1'] == 'S') & (data_2005['env'] == 0)].index) # drop all 'S' except for env = 1

print('\n\n Value Counts, 0 is not env, 1 is env: ')
print(data_2005['env'].value_counts())




 Value Counts, 0 is not env, 1 is env: 
env
0    20182
1     7169
Name: count, dtype: int64


In [93]:
# data import/cleaning for 2017/2018 data
import pandas as pd

file_path = '/Users/colden/Documents/VSCode/Emos_Classification/Environment_Data_2015_2016_cleaned.csv'
data_2017 = pd.read_csv(file_path)
data_2017['data_year'] = 2017

# create ntee1 column
data_2017['ntee1'] = data_2017['nteecc'].str[0]
#data = data[data['ntee1'] != 'S']
#data_2017 = data_2017.drop(data_2017[(data_2017['ntee1'] == 'S') & (data_2017['env'] == 0)].index) # drop all 'S' except for env = 1

data_2017['mission'] = data_2017['mission'].str.replace('SEE SCHEDULE O', '')
data_2017['description'] = data_2017['name'] + ' ' + data_2017['mission']


print('\n\n Value Counts, 0 is not env, 1 is env: ')
print(data_2017['env'].value_counts())



 Value Counts, 0 is not env, 1 is env: 
env
0    21010
1     4763
Name: count, dtype: int64


In [ ]:
# set data

# drop a percent of a given ntee1 for 0 vals only
def drop_perc_ntee1(data, ntee1, percent):
    selected_rows = data[(data['ntee1'] == ntee1) & (data['env'] == 0)]
    drop_rows = selected_rows.sample(frac=percent, random_state=1)
    data_dropped = data.drop(drop_rows.index)
    # print(data[data['ntee1'] == 'S']['env'].value_counts()) # prior to drop
    # print(data_dropped[data_dropped['ntee1'] == 'S']['env'].value_counts()) # after drop
    return data_dropped


data = pd.concat([data_2005, data_2017], ignore_index=True)
data = drop_perc_ntee1(data, 'S', 0.7)
data.reset_index(drop=True, inplace=True) # this NEEDS to be called after drop_perc_ntee1!!! otherwise everything breaks (can't index later on with functions) # wow I'm slow
data.insert(0, 'row_num', data.index)


print(data['env'].value_counts())

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
#nltk.download('stopwords')

data['description'] = data['description'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

stop_words = set(stopwords.words('english'))
stop_words.add('s')
stop_words.add('nan')


# Remove stopwords from the description
data['description'] = data['description'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stop_words]))

# lematize function - taken from IBM: https://www.ibm.com/topics/stemming-lemmatization#:~:text=The%20practical%20distinction%20between%20stemming,be%20found%20in%20the%20dictionary.
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return wordnet.NOUN

def lemmatize_passage(text):
    words = word_tokenize(text)
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_sentence

data['description'] = data['description'].apply(lemmatize_passage)
print("/n/n Lemmatized Data: ")

print(data['description'][0])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

confusion_matrix = pd.crosstab(data['ntee1'], data['env'])

# Create a heatmap from the confusion matrix
plt.figure(figsize=(8, 8))
sns.heatmap(confusion_matrix/len(data), annot=True, cmap='Blues', fmt='.2%')
#sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')

plt.title('NTEE Code and Environmental Organization')
plt.xlabel('Environmental Organization')
plt.ylabel('NTEE Code')

plt.show()


In [ ]:
# Add NTEE codes to the description

# def map_letters(letter):
#     # if letter is not a, b, or c
#     if letter not in ['c', 'b', 's', 'd']:
#         return 'z'
#     else:
#         return letter
        
# data['description'] = data['description'] + ' ' + data['ntee1'].apply(map_letters) * 3


# data['description'] = data['description'] + ' ' + data['ntee1'] * 3 # need to append 3 letters or the tokenizer won't work properly!!! so strange



# map letters to numbers
letter_to_num = {}

# Iterate over each letter in the alphabet
for i, letter in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    # Add the letter to the dictionary with its corresponding number
    letter_to_num[letter] = i + 1

data['ntee_num'] = data['ntee1'].apply(lambda x: letter_to_num.get(x, 0))




In [ ]:
# put all of the cleaned data into a corpus
# need to drop nan values
corpus = []
for i in range(len(data)):
    row = data.iloc[i]
    summary = row['description'].lower()
    summary = summary.split()
    clean_summary = ' '.join(summary)
    corpus.append(clean_summary)
    #print(row['env'])
    

corpus[1]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
input_size = 1500 # number of words to keep -- note that this is also what is fed into the neural network as input size

vectorizer = TfidfVectorizer(max_features=input_size, min_df=3, max_df=0.8) #max features is number of words to keep, min_df is min # of documents a word must appear in, max_df is max percentage of documents a word can appear in
X = vectorizer.fit_transform(corpus).toarray()

y = data.loc[:, 'env'].values # env col



In [ ]:

# add in ntee codes (create dummies, then append these values to X)
dummies = pd.get_dummies(data['ntee_num'], prefix = 'ntee')
print(dummies)

input_size += dummies.shape[1] # add the number of columns to the input size -- note that if this code block is run multiple times, must restart and run all
X = np.concatenate((X, dummies), axis = 1)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

corpus_indices = np.arange(len(corpus))
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, corpus_indices, test_size=0.2, random_state=0)

# How to print training data corresponding to corpus
# print(idx_test[0]) # index of first test data
# print(corpus[idx_test[0]]) # mechanism to print the first test data


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()



In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(input_size) # number of words in the vectorized data
output_size = 2 # binary classification -- maybe needs to be changed to get percentages?
hidden_size = 500 # number of neurons in the hidden layer -- this should be tweaked

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=-1)

model = Net()

import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=0.0005) # drop this down * 10^-1 more? increase epochs
loss_fn = nn.NLLLoss() # CrossEntropyLoss() is the same as NLLLoss() with log_softmax ?



In [ ]:
epochs =  100
losses = []
testing = []
accuracy = None
for epoch in range(epochs):
    optimizer.zero_grad()
    # Y_pred = model(X_train, ntee_train)
    Y_pred = model(X_train)
    loss = loss_fn(Y_pred, y_train.long())
    losses.append(loss.detach().numpy())
    loss.backward()
    optimizer.step()


    if epoch % 10 == 0:
        print(f'Epoch {epoch} / {epochs}, Loss: {loss.item()}')
        print(f'Accuracy on test data at Epoch {epoch}: {accuracy}%')

    model.eval()
    with torch.no_grad():
        # Assuming X_test is your test data and y_test are the true labels
        Y_test_pred = model(X_test)
        # Convert the predictions to class labels
        _, predicted_labels = torch.max(Y_test_pred, 1)

        # Calculate the number of correct predictions
        correct_predictions = (predicted_labels == y_test).sum().item()
        total_predictions = y_test.size(0)
        accuracy = correct_predictions / total_predictions  # Accuracy as a percentage
        testing.append(accuracy)

    # Switch back to training mode
    model.train()


# (21880x1502 and 1500x500)
# so, multiply with 1502 columns and 1502 rows is ok -- input_size vars will be the number of rows

In [ ]:
plt.style.use('ggplot')
# loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(epochs), losses, color = 'red')
plt.ylabel("Loss/Error")
plt.xlabel('Epoch')
plt.title("Loss")

# accuracy
plt.subplot(1, 2, 2)
plt.plot(range(epochs), testing, color = "blue")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title("Accuracy")

plt.tight_layout()
plt.show()

In [ ]:


threshold = 0 # .5 change to 9/53 --> only 2 more identified, but many more cases # 0.2 seems to be pretty good

false_neg_list, false_pos_list, correct_list, uncertain_list = [], [], [], []
false_neg, false_pos, correct, uncertain = 0, 0, 0, 0
y_predictions = []
y_actual = []
true_pos, true_neg = 0, 0 # sum to 'correct'

with torch.no_grad():
   for i, X_test_row in enumerate(X_test):
      y_val = model.forward(X_test_row)
      y_predictions.append(y_val)
      predicted_class = y_val.argmax().item()
      actual_class = y_test[i].item()
      y_actual.append(actual_class)

      if abs(y_val[0].item() - y_val[1].item()) < threshold:
            print(f'{i+1}. {str(y_val)}, predicted {predicted_class}, actual {actual_class}')
            uncertain_list.append(i)
            uncertain += 1
      elif predicted_class == actual_class:
         correct_list.append(i)
         correct += 1
         if predicted_class == 0:
            true_neg += 1
         else:
            true_pos += 1
      elif predicted_class != actual_class:
         if predicted_class == 0:
            false_neg_list.append(i)
            false_neg += 1
         else:
            false_pos_list.append(i)
            false_pos += 1



print(f'Correct: {correct}, False: {false_neg + false_pos}, Uncertain: {uncertain}')
print(f'False prediction 0: {false_neg}, False prediction 1: {false_pos}')

print(f'Accuracy: {(correct + uncertain) / (correct + false_neg + false_pos + uncertain)}')
print(f'Precision: {true_pos / (true_pos + false_pos)}')
print(f'Recall: {true_pos / (true_pos + false_neg)}')

# currently, TfidfVectorizer is used to train the model. However, this is a bag of words model, which does not take into account the context of the words.
# This is a limitation of the model. A better model might be a word2vec, BERT, FastText, Glove, etc. model, which takes into account the context of the words.
# would also be nice to see which tensor goes with which word

In [ ]:
# predictions for the ntee model
np.set_printoptions(threshold=500, linewidth=200, edgeitems=30)

# given a test index, return the corresponding row in 'data'
def get_from_testdata(position):
    row = data.loc[idx_test[position]] # get the row in 'data' corresponding to the test index
    return row

# print(idx_test[0])
# print(data.loc[20050])
# get_datarow(0)
# print()

# input a row from 'data' to convert to tensor (with ntee codes)
def predict_nteeincl(row_index):
    input_text = vectorizer.transform([data['description'][row_index]]).toarray()
    dummies_row = dummies.loc[row_index].values.reshape(1,-1) # reshape dummies list to 2d array 
    input_matrix = np.concatenate((input_text, dummies_row), axis = 1) # combine dummies (ntee) and input text
    input_matrix = torch.from_numpy(input_matrix).float()
    output = model(input_matrix)
    return str(output) + ' , predicted:' + str(output.argmax().item())

# print(predict_nteeincl(0)) # get prediction

# pass in an array of indices, return the corresponding rows in 'data'
def return_rows(array):
    df_list = []
    for test_i in array:
        data_i = get_from_testdata(test_i)
        df_list.append(data_i)
    return_df = pd.concat(df_list, axis=1).transpose()
    return return_df



In [ ]:
import os
directory = 'withNTEEoutputdirectory'
if not os.path.exists(directory):
    os.makedirs(directory)

return_rows(false_neg_list).to_csv(f'{directory}/false_neg.csv', index=False)
return_rows(false_pos_list).to_csv(f'{directory}/false_pos.csv', index=False)
return_rows(correct_list).to_csv(f'{directory}/correct.csv', index=False)
try:
    return_rows(uncertain_list).to_csv(f'{directory}/uncertain.csv', index=False)
except ValueError:
    print('No uncertain predictions')

# create prediction_result column in 'data' to show prediction results
def update_prediction_result(position_list, result):
    for position in position_list:
        index = idx_test[position]
        data.loc[index, 'Prediction_Result'] = result

update_prediction_result(correct_list, "correct")
update_prediction_result(false_neg_list, "false_neg")
update_prediction_result(false_pos_list, "false_pos")
update_prediction_result(uncertain_list, "uncertain")

data.to_csv(f'{directory}/final_data.csv')

In [ ]:
confusion_matrix = pd.crosstab(data['ntee1'], data['Prediction_Result'])

# Create a heatmap from the confusion matrix
plt.figure(figsize=(8, 8))
sns.heatmap(confusion_matrix/len(y_test), annot=True, cmap='Blues', fmt='.2%')
# sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')

plt.title('NTEE Code and Category Prediction (%)')
plt.xlabel('Prediction')
plt.ylabel('NTEE Code')

plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')

plt.title('NTEE Code and Category Prediction (Numeric)')
plt.xlabel('Prediction')
plt.ylabel('NTEE Code')

plt.show()

In [ ]:
print(data.keys())

confusion_matrix_predictedvactual = pd.crosstab(data['env'], data['Prediction_Result'])

# Create a heatmap from the confusion matrix
plt.figure(figsize=(8, 8))
sns.heatmap(confusion_matrix_predictedvactual, annot=True, cmap='Blues', fmt='g')
# sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')

plt.title('Predicted vs. Actual (NTEE model)')
plt.xlabel('Prediction')
plt.ylabel('Actual Value')

plt.show()